In [1]:
import gurobipy as gp
from gurobipy import GRB

# 示例数据

stock_length = 100  # 每个原材料的长度
demand = [15, 25, 30, 40, 50]  # 客户的需求长度
demand_num = [10 for i in range(len(demand))]  # 需求数量，确保长度与demand一致

# 组合成字典，保证每个demand对应一个demand_num
demand_dict = {d: n for d, n in zip(demand, demand_num)}

# 循环打印字典
for d, n in demand_dict.items():
    print(f"Demand Length = {d}, Demand Num = {n}")


cut_patterns = [  # 每个切割模式的长度配置
    [15, 15,15,25, 30],  # 第1种模式，使用长度为15, 25, 30的切割
    [40, 50],      # 第2种模式，使用长度为40, 50的切割
    [15,15,30,40 ],      # 第3种模式，使用长度为30, 70的切割
    [25,25,40 ],      # 第4种模式，使用长度为60, 40的切割
    [30,30,40],  # 第5种模式，使用长度为10, 10, 30的切割
]






Demand Length = 15, Demand Num = 10
Demand Length = 25, Demand Num = 10
Demand Length = 30, Demand Num = 10
Demand Length = 40, Demand Num = 10
Demand Length = 50, Demand Num = 10


In [2]:
import gurobipy as gp
from gurobipy import GRB

def solve_cut_whole(demand_dict, stock_num,stock_length):
    model = gp.Model("cutting_stock")
    
    # 创建变量 x[i][j] (每个切割模式 i 和每个需求 j 对应的变量)
    x = [[model.addVar(vtype=GRB.INTEGER, name=f"x_{i}_{j}") 
          for j in range(len(demand_dict))] 
         for i in range(stock_num)]
    
    # 创建变量 y[i] (每个切割模式是否使用)
    y = model.addVars(stock_num, vtype=GRB.BINARY, name="y")  
    
    # 目标函数: 最小化使用的切割模式数量
    model.setObjective(gp.quicksum(y[i] for i in range(stock_num)), GRB.MINIMIZE)

    # 添加约束：对于每个切割模式，限制它的总使用量不超过 100 * y[i]
    for i in range(stock_num):
        lensum = gp.LinExpr()
        for idx, (d, n) in enumerate(demand_dict.items()):  
            lensum+= x[i][idx] * d
        model.addConstr(
            lensum<=stock_length*y[i],
            name=f"demand_stock_{i}"
        )

    # 添加约束：满足每个客户的需求
    for idx, (d, n) in enumerate(demand_dict.items()):  
        model.addConstr(
            gp.quicksum(x[i][idx] for i in range(stock_num)) >= n,
            name=f"demand_{d}"
        )

    # 求解模型
    model.optimize()
    
    # 输出结果
    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:")
        for i in range(stock_num):
            # if y[i].x>0:
            #     print(f"y_{i}: {y[i].x}")
            for j in range(len(demand_dict)):
                x_value = x[i][j].x
                if x_value > 0:  # 如果 x[i][j] 的值大于0，打印它
                    print(f"x_{i}_{j}: {x_value}")
    else:
        print("No optimal solution found")


stock_num = 30  

solve_cut_whole(demand_dict, stock_num,100)


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 35 rows, 180 columns and 330 nonzeros
Model fingerprint: 0x1a3eb704
Variable types: 0 continuous, 180 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve time: 0.00s
Presolved: 35 rows, 180 columns, 330 nonzeros
Variable types: 0 continuous, 180 integer (30 binary)
Found heuristic solution: objective 19.0000000

Root relaxation: objective 1.600000e+01, 85 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   16.00000    0    9   19.00000   16.00000  15.8%     -

In [3]:
cut_patterns

[[15, 15, 15, 25, 30], [40, 50], [15, 15, 30, 40], [25, 25, 40], [30, 30, 40]]

In [4]:
demand_dict

{15: 10, 25: 10, 30: 10, 40: 10, 50: 10}

In [5]:
def solve_cut(cut_patterns,demand_dict):
    model = gp.Model("cutting_stock")
    x = model.addVars(len(cut_patterns), vtype=GRB.INTEGER, name="x")  # pattern_num

    #objective_function
    model.setObjective(gp.quicksum(x[i] for i in range(len(cut_patterns))), GRB.MINIMIZE)

    # satisfy demand
    for d, n in demand_dict.items():
        model.addConstr(
            gp.quicksum(x[j] * cut_patterns[j].count(d) for j in range(len(cut_patterns))) >= n,
            name=f"demand_{d}"
        )
    
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:")
        for v in model.getVars():
            if v.x > 0:  
                print(f"{v.varName}: {v.x}")
                pattern_index = int(v.varName.split('[')[-1].split(']')[0]) 
                print(f"Pattern {pattern_index}: {cut_patterns[pattern_index]}, x = {v.x}")
    else:
        print("No optimal solution found")
solve_cut(cut_patterns=cut_patterns,demand_dict=demand_dict)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 5 rows, 5 columns and 12 nonzeros
Model fingerprint: 0xe2dc02dd
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 23.0000000
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 2: 20 23 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%

Optimal solution found:
x[0]: 10.0
Pattern 0: [15, 15, 15, 25, 30

In [6]:
solve_cut(cut_patterns=cut_patterns,demand_dict=demand_dict)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 5 rows, 5 columns and 12 nonzeros
Model fingerprint: 0xe2dc02dd
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 23.0000000
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 2: 20 23 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%

Optimal solution found:
x[0]: 10.0
Pattern 0: [15, 15, 15, 25, 30

In [7]:
list(demand_dict.values())

[10, 10, 10, 10, 10]

In [8]:
import numpy as np
def solve_dual(cut_patterns,demand_dict):
    model = gp.Model("cutting_stock")
    x = model.addVars(len(demand_dict), vtype=GRB.INTEGER, name="x")  # pattern_num

    #objective_function
    model.setObjective(gp.quicksum(list(demand_dict.values())[i]*x[i] for i in range(len(demand_dict))), GRB.MAXIMIZE)


    A = np.zeros((len(demand_dict), len(cut_patterns)))

    # 填充矩阵 A
    for i, (d, n) in enumerate(demand_dict.items()):
        for j, pattern in enumerate(cut_patterns):
            A[i, j] = pattern.count(d)  # 这里是 count(d)，也可以换成其他的需求计算方法
        
    AT = A.T
    for i in range(len(cut_patterns)):
        cargosum = gp.LinExpr()
        for j in range(len(demand_dict)):
            cargosum+=AT[i,j]*x[j]
        model.addConstr(cargosum<=1)
    

    model.optimize()
    returnlist = []
    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:")
        for v in model.getVars():  
            print(f"{v.varName}: {v.x}")
            returnlist.append(v.x)
    else:
        print("No optimal solution found")
    return returnlist

In [9]:
dual = solve_dual(cut_patterns=cut_patterns,demand_dict=demand_dict)
dual

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 5 rows, 5 columns and 12 nonzeros
Model fingerprint: 0x91ab840c
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%

Optimal solution found:
x[0]: 0.0
x[1]: 0.0
x[2]: 0.0
x[3]: 1.0
x[4]

[0.0, 0.0, 0.0, 1.0, -0.0]

In [10]:
import gurobipy as gp
from gurobipy import GRB

def solve_diedai(dual_s, stock_length, demand_dict, cut_patterns):
    # 创建模型
    model = gp.Model("cutting_stock")
    
    # 添加决策变量
    x = model.addVars(len(demand_dict), vtype=GRB.INTEGER, name="x")  # pattern_num

    # 设置目标函数
    model.setObjective(1 - gp.quicksum(dual_s[i] * x[i] for i in range(len(demand_dict))), GRB.MINIMIZE)

    # 添加约束：每种需求的总切割长度不能超过 stock_length
    for d, n in demand_dict.items():
        model.addConstr(
            gp.quicksum(x[j] * d for j in range(len(demand_dict))) <= stock_length,
            name=f"demand_{d}"
        )
        
    # 求解模型
    model.optimize()
    
    returnlist = []
    if model.status == GRB.OPTIMAL:
        # 如果有最优解，检查目标值是否小于 0
        if model.objVal < 0:
            print("\nOptimal solution found with negative objective value:")
            for v in model.getVars():  
                print(f"{v.varName}: {v.x}")
                returnlist.append(v.x)
            return returnlist, True
        else:
            # 如果最优值不小于 0
            print("Objective value is not negative.")
            return [], False
    else:
        # 如果没有最优解
        print("No optimal solution found")
        return [], False


In [11]:
l1,cs = solve_diedai(dual,stock_length,demand_dict,cut_patterns)
cs

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 5 rows, 5 columns and 25 nonzeros
Model fingerprint: 0x8154eca8
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+01, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective -1.0000000
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 1: -1 

Optimal solution found (tolerance 1.00e-04)
Best objective -1.000000000000e+00, best bound -1.000000000000e+00, gap 0.0000%

Optimal solution found with negative objective value:
x[0]: -0.0
x

True

In [12]:
def col(cut_patterns,demand_dict,stock_length):
    cs = True
    num = 0
    print("first solve")
    solve_cut(cut_patterns=cut_patterns,demand_dict=demand_dict)
    dual = solve_dual(cut_patterns=cut_patterns,demand_dict=demand_dict)
    l1,cs = solve_diedai(dual,stock_length,demand_dict,cut_patterns)
    while cs:
        l2 = []
        for i in range(len(l1)):
            if l1[i]==0:
                continue
            else:
                l2.extend([list(demand_dict.keys())[i] for j in range(int(l1[i]))])
        cut_patterns.append(l2)
        num+=1
        print(cut_patterns)
        print(f"{num}-th solve")
        solve_cut(cut_patterns=cut_patterns,demand_dict=demand_dict)
        dual = solve_dual(cut_patterns=cut_patterns,demand_dict=demand_dict)
        l1,cs = solve_diedai(dual,stock_length,demand_dict,cut_patterns)

In [13]:
col(cut_patterns=cut_patterns,demand_dict=demand_dict,stock_length=100)

first solve
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 5 rows, 5 columns and 12 nonzeros
Model fingerprint: 0xe2dc02dd
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 23.0000000
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 2: 20 23 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%

Optimal solution found:
x[0]: 10.0
Pattern 0: [15, 15

In [14]:
import matplotlib.pyplot as plt

In [15]:
import numpy as np
np.version

<module 'numpy.version' from 'C:\\Users\\19513\\AppData\\Roaming\\Python\\Python39\\site-packages\\numpy\\version.py'>